# Create k-mer matrix

This notebook creates the k-mer count matrix for 
N. gonorrhoeae
Azithromycin

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [96]:
import numpy as np
import pandas as pd
import pylab as plt
import glob
import os
from scipy.sparse import lil_matrix

### Load k-mers from DSK output

In [10]:
data_dir = 'C:\\Users\\joewa\\Downloads\\kmersOutAscii'

In [19]:
filenames = sorted(glob.glob(os.path.join(data_dir, '*.out')))

In [58]:
dfs = []
for i in range(len(filenames)):
    filename = filenames[i]
    
    # load the output from dsk2ascii
    print('%d/%d %s' % (i+1, len(filenames), filename))
    df = pd.read_csv(filename, header=None, sep=' ')
    df = df.rename(columns={0: 'kmer', 1: 'count'})
    
    # get strain information from filename
    basename = os.path.basename(filename)
    strain = '.'.join(basename.split('.')[0:-2])
    df['strain'] = strain
    
    # reorder columns
    cols = ['strain', 'kmer', 'count']
    df = df[cols]
    
    dfs.append(df)

1/391 C:\Users\joewa\Downloads\kmersOutAscii\485.291.h5.out
2/391 C:\Users\joewa\Downloads\kmersOutAscii\485.292.h5.out
3/391 C:\Users\joewa\Downloads\kmersOutAscii\485.294.h5.out
4/391 C:\Users\joewa\Downloads\kmersOutAscii\485.295.h5.out
5/391 C:\Users\joewa\Downloads\kmersOutAscii\485.298.h5.out
6/391 C:\Users\joewa\Downloads\kmersOutAscii\485.299.h5.out
7/391 C:\Users\joewa\Downloads\kmersOutAscii\485.301.h5.out
8/391 C:\Users\joewa\Downloads\kmersOutAscii\485.305.h5.out
9/391 C:\Users\joewa\Downloads\kmersOutAscii\485.307.h5.out
10/391 C:\Users\joewa\Downloads\kmersOutAscii\485.311.h5.out
11/391 C:\Users\joewa\Downloads\kmersOutAscii\485.312.h5.out
12/391 C:\Users\joewa\Downloads\kmersOutAscii\485.313.h5.out
13/391 C:\Users\joewa\Downloads\kmersOutAscii\485.314.h5.out
14/391 C:\Users\joewa\Downloads\kmersOutAscii\485.316.h5.out
15/391 C:\Users\joewa\Downloads\kmersOutAscii\485.317.h5.out
16/391 C:\Users\joewa\Downloads\kmersOutAscii\485.318.h5.out
17/391 C:\Users\joewa\Downloads\k

138/391 C:\Users\joewa\Downloads\kmersOutAscii\485.526.h5.out
139/391 C:\Users\joewa\Downloads\kmersOutAscii\485.527.h5.out
140/391 C:\Users\joewa\Downloads\kmersOutAscii\485.528.h5.out
141/391 C:\Users\joewa\Downloads\kmersOutAscii\485.529.h5.out
142/391 C:\Users\joewa\Downloads\kmersOutAscii\485.530.h5.out
143/391 C:\Users\joewa\Downloads\kmersOutAscii\485.531.h5.out
144/391 C:\Users\joewa\Downloads\kmersOutAscii\485.532.h5.out
145/391 C:\Users\joewa\Downloads\kmersOutAscii\485.533.h5.out
146/391 C:\Users\joewa\Downloads\kmersOutAscii\485.534.h5.out
147/391 C:\Users\joewa\Downloads\kmersOutAscii\485.535.h5.out
148/391 C:\Users\joewa\Downloads\kmersOutAscii\485.536.h5.out
149/391 C:\Users\joewa\Downloads\kmersOutAscii\485.537.h5.out
150/391 C:\Users\joewa\Downloads\kmersOutAscii\485.538.h5.out
151/391 C:\Users\joewa\Downloads\kmersOutAscii\485.539.h5.out
152/391 C:\Users\joewa\Downloads\kmersOutAscii\485.540.h5.out
153/391 C:\Users\joewa\Downloads\kmersOutAscii\485.541.h5.out
154/391 

272/391 C:\Users\joewa\Downloads\kmersOutAscii\485.753.h5.out
273/391 C:\Users\joewa\Downloads\kmersOutAscii\485.754.h5.out
274/391 C:\Users\joewa\Downloads\kmersOutAscii\485.755.h5.out
275/391 C:\Users\joewa\Downloads\kmersOutAscii\485.756.h5.out
276/391 C:\Users\joewa\Downloads\kmersOutAscii\485.757.h5.out
277/391 C:\Users\joewa\Downloads\kmersOutAscii\485.758.h5.out
278/391 C:\Users\joewa\Downloads\kmersOutAscii\485.759.h5.out
279/391 C:\Users\joewa\Downloads\kmersOutAscii\485.760.h5.out
280/391 C:\Users\joewa\Downloads\kmersOutAscii\485.761.h5.out
281/391 C:\Users\joewa\Downloads\kmersOutAscii\485.762.h5.out
282/391 C:\Users\joewa\Downloads\kmersOutAscii\485.763.h5.out
283/391 C:\Users\joewa\Downloads\kmersOutAscii\485.764.h5.out
284/391 C:\Users\joewa\Downloads\kmersOutAscii\485.765.h5.out
285/391 C:\Users\joewa\Downloads\kmersOutAscii\485.766.h5.out
286/391 C:\Users\joewa\Downloads\kmersOutAscii\485.767.h5.out
287/391 C:\Users\joewa\Downloads\kmersOutAscii\485.768.h5.out
288/391 

In [59]:
df = pd.concat(dfs)

In [80]:
df

,strain,kmer,count
0,485.291,AAAAAAAAAAAAAAAAAAAAA,43
1,485.291,AAAAAAACCGAAACCGAACGG,2
2,485.291,AAAAAACAATAAACTATAACA,2
3,485.291,AAAAAAGGCATTTTTTAAATG,5
4,485.291,AAAAACAAAATAGTACAACAC,8
5,485.291,AAAAACAACCGAAACCGGACA,2
6,485.291,AAAAACAATAAACTATAACAA,2
7,485.291,AAAAACAAGCTTAATTTCAGA,2
8,485.291,AAAAACCCAAAAGACGGAACA,4
9,485.291,AAAAATTTTTAACTTTAAAAC,2


### Construct a feature matrix

In [73]:
strains = sorted(df['strain'].unique())
kmers = sorted(df['kmer'].unique())

In [74]:
n_strains = len(strains)
n_kmers = len(kmers)

In [78]:
strain_idx = { k: v for v, k in list(enumerate(strains)) }
kmer_idx = { k: v for v, k in list(enumerate(kmers)) }

In [97]:
feature = lil_matrix((n_strains, n_kmers))

In [100]:
feature.shape

(391, 354280)

In [101]:
feature

<391x354280 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in LInked List format>

In [ ]:
total = df.shape[0]
counter = 0
for idx, row in df.iterrows():
    counter += 1
    if counter % 100000 == 0:
        progress = counter / total * 100
        print('%.1f%%' % progress)    
    
    strain = row['strain']
    kmer = row['kmer']
    count = row['count']
            
    i = strain_idx[strain]
    j = kmer_idx[kmer]
    feature[i, j] = count

### Load target prediction

In [113]:
predicted_df = pd.read_csv('C:\\Users\\joewa\\Downloads\\metadata.tsv.txt', sep='\t', header=None)
predicted_df = predicted_df.rename(columns={0: 'strain', 1: 'resistance'})

In [115]:
predicted_df.set_index('strain')

,resistance
strain,
485.874,sensitive
485.757,sensitive
485.409,sensitive
485.756,sensitive
485.755,sensitive
485.754,sensitive
485.753,sensitive
485.751,sensitive
485.415,sensitive
